# EMG Preprocessing

## Load Raw EMG Data

In [ ]:
import pandas as pd

data = pd.read_csv('emg_data.csv')
sampling_frequency = 1000 # Hz

## Filtering Functions

In [ ]:
from scipy.signal import butter, filtfilt

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_notch(cutoff, fs, Q=30):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(2, [normal_cutoff - normal_cutoff/Q, normal_cutoff + normal_cutoff/Q], btype='bandstop')
    return b, a

def filter_signal(data, fs):
    b, a = butter_highpass(20, fs, order=4)
    highpassed = filtfilt(b, a, data)
    
    b, a = butter_lowpass(450, fs, order=4)
    lowpassed = filtfilt(b, a, highpassed)
    
    b, a = butter_notch(50, fs)
    filtered = filtfilt(b, a, lowpassed)
    
    return filtered

# Define Preprocessing Function

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def preprocess_emg(data, fs):
    # Filtering
    filtered_data = filter_signal(data, fs)
    
    # Rectification
    rectified_data = np.abs(filtered_data)
    
    # Smoothing
    window_size = int(0.2 * fs)  # 200 ms window
    smoothed_data = np.convolve(rectified_data, np.ones(window_size)/window_size, mode='same')
    
    # Normalization
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(smoothed_data.reshape(-1, 1)).flatten()
    
    return normalized_data

## Apply Preprocessing To Each EMG Channel

In [ ]:
emg_channels = ['EMG1', 'EMG2', 'EMG3', 'EMG4', 'EMG5', 'EMG6', 'EMG7', 'EMG8']
preprocessed_data = {}

for channel in emg_channels:
    preprocessed_data[channel] = preprocess_emg(data[channel].values, sampling_frequency)

preprocessed_df = pd.DataFrame(preprocessed_data)

## Plot Preprocessed EMG Data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(data['EMG1'])
plt.title('Raw EMG Data - EMG1')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(preprocessed_df['EMG1'])
plt.title('Preprocessed EMG Data - EMG1')
plt.xlabel('Time (samples)')
plt.ylabel('Normalized Amplitude')

plt.tight_layout()
plt.show()